Nama : Aditya Atadewa  
Kelas : TI 3G  
NIM : 2341720174  
Absen : 01  

# Praktikum 3 - HNSW

In [1]:
!pip install hnswlib -q

In [2]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[497 206 346 595 875]]
Distances: [[0.01247727 0.02432784 0.03830282 0.03840221 0.04826941]]
Waktu: 0.024801015853881836 detik

=== HNSW ===
Indices: [[497 206 346 595 875]]
Distances: [[0.00015568 0.00059184 0.00146711 0.00147473 0.00232994]]
Waktu: 0.00016164779663085938 detik


## Percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data.

In [3]:
import hnswlib
import numpy as np
import time
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from tabulate import tabulate

# ==============================================
# Fungsi bantu untuk satu percobaan
# ==============================================
def experiment_hnsw_vs_exact(n_points, dim, metric):
    """
    Menjalankan satu eksperimen perbandingan HNSW vs Exact NN.
    """
    print(f"Menjalankan eksperimen: {n_points} data, {dim}D, metric={metric.upper()}")

    # --- 1. Buat dataset acak ---
    np.random.seed(42)
    data = np.random.rand(n_points, dim).astype(np.float32)
    query = np.random.rand(1, dim).astype(np.float32)
    k = 5

    # --- 2. Normalisasi jika menggunakan Inner Product ---
    if metric == 'ip':
        data /= np.linalg.norm(data, axis=1, keepdims=True)
        query /= np.linalg.norm(query, axis=1, keepdims=True)
        sklearn_metric = 'cosine'
    else:
        sklearn_metric = 'euclidean'

    # --- 3. Exact NN (Brute Force) ---
    nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=sklearn_metric)
    nn.fit(data)

    t0 = time.time()
    _, idx_exact = nn.kneighbors(query)
    t_exact = time.time() - t0

    # --- 4. Approximate NN dengan HNSW ---
    index = hnswlib.Index(space=metric, dim=dim)

    # Build index (train + add)
    t0 = time.time()
    index.init_index(max_elements=n_points, ef_construction=200, M=16)
    index.add_items(data)
    build_time = time.time() - t0

    # Search
    index.set_ef(50)
    t0 = time.time()
    idx_hnsw, _ = index.knn_query(query, k=k)
    t_hnsw = time.time() - t0

    # --- 5. Hitung Recall (akurasi approximate) ---
    recall = len(set(idx_exact[0]) & set(idx_hnsw[0])) / k

    return {
        "Ukuran Data": n_points,
        "Dimensi": dim,
        "Metrik Jarak": metric.upper(),
        "Waktu Build (HNSW) (s)": round(build_time, 6),
        "Waktu Search (Exact) (s)": round(t_exact, 6),
        "Waktu Search (HNSW) (s)": round(t_hnsw, 6),
        f"Recall@{k}": round(recall, 2)
    }

# ==============================================
# Daftar skenario percobaan
# ==============================================
scenarios = [
    (1000, 2, 'l2'),
    (1000, 5, 'l2'),
    (1_000_000, 2, 'l2'),
    (1_000_000, 5, 'l2'),
    (1000, 2, 'ip'),
    (1000, 5, 'ip'),
    (1_000_000, 2, 'ip'),
    (1_000_000, 5, 'ip'),
]

# ==============================================
# Jalankan semua percobaan
# ==============================================
results = []
for n, d, m in scenarios:
    results.append(experiment_hnsw_vs_exact(n, d, m))

# ==============================================
# Tampilkan hasil dalam bentuk tabel
# ==============================================
df = pd.DataFrame(results)
print("\n=== HASIL EKSPERIMEN HNSW vs EXACT NN ===\n")
print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))


Menjalankan eksperimen: 1000 data, 2D, metric=L2
Menjalankan eksperimen: 1000 data, 5D, metric=L2
Menjalankan eksperimen: 1000000 data, 2D, metric=L2
Menjalankan eksperimen: 1000000 data, 5D, metric=L2
Menjalankan eksperimen: 1000 data, 2D, metric=IP
Menjalankan eksperimen: 1000 data, 5D, metric=IP
Menjalankan eksperimen: 1000000 data, 2D, metric=IP
Menjalankan eksperimen: 1000000 data, 5D, metric=IP

=== HASIL EKSPERIMEN HNSW vs EXACT NN ===

+---------------+-----------+----------------+--------------------------+----------------------------+---------------------------+------------+
|   Ukuran Data |   Dimensi | Metrik Jarak   |   Waktu Build (HNSW) (s) |   Waktu Search (Exact) (s) |   Waktu Search (HNSW) (s) |   Recall@5 |
+===============+===========+================+==========================+============================+===========================+============+
|          1000 |         2 | L2             |                 0.055089 |                   0.001229 |                  